In [13]:
import re
import unicodedata
import pandas as pd
import numpy as np
import nltk
import os
import nltk.corpus
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
corpus = set(stopwords.words('english'))
nltk.download('words')

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\words.zip.


True

In [3]:
def remove_code_blocks(string):
    list_block = string.split('```')
    new_list_block = []
    for index, item in enumerate(list_block):
        if index % 2 == 0:
            new_list_block.append(item)
    
    return ' '.join(new_list_block)

In [4]:
def nlp_basic_clean(string):
    """
    Lowercases, removes non-ASCII characters, and removes non-alphanumeric (except ' or \s') from the passed in string.
    """
    
    cleaned_string = string
    
    cleaned_string = cleaned_string.lower()
    cleaned_string = unicodedata.normalize("NFKD", cleaned_string).encode("ascii", "ignore").decode("utf-8", "ignore")
    cleaned_string = re.sub('<[^<]+?>', ' ', cleaned_string)
    cleaned_string = re.sub(r"\n", " ", cleaned_string)
    cleaned_string = re.sub(r'http\S+', ' ', cleaned_string)
    cleaned_string = remove_code_blocks(cleaned_string)
    cleaned_string = re.sub(r"[^a-z'\s]", ' ', cleaned_string)
    cleaned_string = re.sub(r"\s\s+" , ' ', cleaned_string)
    cleaned_string = cleaned_string.strip()
    return cleaned_string

In [5]:
def read_text_file(file_path):
    file = open(file_path, mode='r', encoding="utf8")
    all_of_it = file.read()
    file.close()
    return all_of_it

In [6]:
def tokenize_text(string):
    token_list = word_tokenize(string)
    return token_list

In [7]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer() 

def text_lemmatizing(val):
    datas = val
    for index, data in enumerate(datas):
        datas[index] = lemmatizer.lemmatize(data)
    return datas


In [8]:
def remove_stopwords(tokenize_list):
    tokenize_list = [x for x in tokenize_list if x not in corpus]
    return tokenize_list


In [9]:
from nltk.probability import FreqDist

def freq_text(token_list):
    fdist = FreqDist(token_list)
    return fdist

def freq_perform(fdist):
    print(max(fdist), max(fdist.values()))

    fdist.plot()

In [109]:
from nltk.corpus import words

def remove_singleton_doubleton(tokenize_list):
    return [ i for i in tokenize_list if len(i) > 3 ]

def remove_non_english_word(tokenize_list):
    corpus_words = words.words()
    return [ i for i in tokenize_list if i in corpus_words ]


In [110]:
def get_final_clean_text(file_path):
    text = read_text_file(file_path)
    clean_text = nlp_basic_clean(text)
    tokenize_list = tokenize_text(clean_text)
    tokenize_list = text_lemmatizing(tokenize_list)
    tokenize_list = remove_stopwords(tokenize_list)
    tokenize_list = remove_singleton_doubleton(tokenize_list)
    tokenize_list = remove_non_english_word(tokenize_list)
    final_clean_text = ' '.join(tokenize_list)
    return final_clean_text



In [111]:
text1 = get_final_clean_text('process/test1.txt')
text2 = get_final_clean_text('process/test2.txt')
text3 = get_final_clean_text('process/test3.txt')
text4 = get_final_clean_text('process/test4.txt')
text5 = get_final_clean_text('process/test5.txt')


In [81]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

In [112]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform([text1, text2, text3, text4, text5])

In [113]:
cosine_sim_2 = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [114]:
cosine_sim_2

array([[1.        , 0.0426953 , 0.15621225, 0.03170753, 0.1216691 ],
       [0.0426953 , 1.        , 0.04657382, 0.        , 0.0386873 ],
       [0.15621225, 0.04657382, 1.        , 0.11660882, 0.30921236],
       [0.03170753, 0.        , 0.11660882, 1.        , 0.18091289],
       [0.1216691 , 0.0386873 , 0.30921236, 0.18091289, 1.        ]])